In [1]:
import pandas as pd

df = pd.read_parquet("/home/work/deeptext/yys/redpen/data/1M-GPT4-Augmented.parquet")
# df = df.to_json(orient="records")

ModuleNotFoundError: No module named 'pandas'

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-large")

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/580 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def calc_len(st):
    return len(tokenizer.tokenize(st))

In [ ]:
df.to_json('/home/work/deeptext/yys/redpen/data/1M-GPT4-Augmented.json',orient="columns")

In [ ]:
import json

with open("/home/work/deeptext/yys/redpen/data/1M-GPT4-Augmented.json", "r") as f:

    data = json.load(f)

# id, system_prompt, question, response

In [ ]:
len(data['response'])

994896

In [ ]:
data['question']['0']

"You will be given a definition of a task first, then some input of the task.\nThis task is about using the specified sentence and converting the sentence to Resource Description Framework (RDF) triplets of the form (subject, predicate object). The RDF triplets generated must be such that the triplets accurately capture the structure and semantics of the input sentence. The input is a sentence and the output is a list of triplets of the form [subject, predicate, object] that capture the relationships present in the sentence. When a sentence has more than 1 RDF triplet possible, the output must contain all of them.\n\nAFC Ajax (amateurs)'s ground is Sportpark De Toekomst where Ajax Youth Academy also play.\nOutput:"

In [ ]:
from tqdm import tqdm

res_list = []
for i in tqdm(range(len(data['question']))):
    tmp_dict = {}
    tmp_dict['idx'] = i
    tmp_dict['question'] = None
    tmp_dict['response'] = None
    idx = str(i)
    q_len = calc_len(data['question'][idx])
    r_len = calc_len(data['response'][idx])
    if q_len+r_len >200:
        continue
    else:
        tmp_dict['question'] = data['question'][idx]
        tmp_dict['response'] = data['response'][idx]
        res_list.append(tmp_dict)

    

100%|██████████| 994896/994896 [29:50<00:00, 555.58it/s]


In [ ]:
len(res_list)

386461

In [ ]:
import json

with open("/home/work/deeptext/yys/redpen/data/gpt4_augmented_length_200.json", "w", encoding='utf-8') as f:
    json.dump(res_list,f , ensure_ascii=False,indent=4)

In [ ]:
import json
with open("/home/work/deeptext/yys/redpen/data/gpt4_augmented_length_200.json", "r") as f:

    data = json.load(f)

In [ ]:
for idx, datum in enumerate(data):
    datum['idx'] = idx

In [ ]:
def is_eng(text):
    pattern = re.compile(r"[^a-zA-Z0-9!!\"#$%&'()*+,-./:;<=>?@\[\]\s]")
    t = re.sub(pattern, '',text)
    if len(text)==len(t):
        return True
    else:
        return False

In [ ]:
from tqdm import  tqdm
from random import  *
import re

new_data = list()

i = 0
while len(new_data)<10000:
    tnum = randint(1,386460)
    if is_eng(data[tnum]['question']) and is_eng(data[tnum]['response']):
        tdict = dict()
        tdict['idx'] = i
        tdict['question'] = None
        tdict['gpt-4'] = {'response':None, 
                        'grammar':None,
                        'completeness':None,
                        'Readability':None}
        tdict['gpt-3.5'] = {'response':None, 
                        'grammar':None,
                        'completeness':None,
                        'Readability':None}
        tdict['flan-paca'] = {'response':None, 
                        'grammar':None,
                        'completeness':None,
                        'Readability':None}
        tdict['dolly-v2'] = {'response':None, 
                        'grammar':None,
                        'completeness':None,
                        'Readability':None}
        tdict['question']=data[tnum]['question']
        tdict['gpt-4']['response']=data[tnum]['response']
        new_data.append(tdict)
        print(i, end='\r')
        i+=1
    else:
        continue
        
    

In [ ]:
len(new_data)
# new_data

10000

In [ ]:
import json

with open("/home/work/deeptext/yys/redpen/data/data_sampled_10k_only_eng.json", "w", encoding='utf-8') as f:
    json.dump(new_data,f , ensure_ascii=False,indent=4)

In [ ]:
import json
with open("/home/work/deeptext/yys/redpen/data/data_sampled_10k_only_eng_dolly_finished.json", "r") as f:

    dolly = json.load(f)

In [ ]:
dolly[0]['dolly_v2']

{'response': '...................... 4 "Like"................................ 3 "Neutral".......................... 2 "Dislike".......................... 1 "Hate"............................. 0  Thank you for your feedback. We appreciate it!  ',
 'grammar': None,
 'completeness': None,
 'readability': None}

In [ ]:
dolly[0]

{'idx': 0,
 'question': 'On a scale of 1-5 (with 1 being least favorable and 5 being most favorable), how would you rate this review? "Love" ',
 'q_len': 30,
 'gpt-4': {'response': '2',
  'grammar': None,
  'completeness': None,
  'readability': None},
 'llama2': {'response': "Thank you for sharing your feedback with us! We are thrilled to hear that you had an amazing experience at our restaurant. Your positive review is greatly appreciated, and we will continue to work hard to provide excellent service and food quality to all of our guests. If there's anything else we can assist you with, please don't hesitate to reach out.",
  'grammar': None,
  'completeness': None,
  'readability': None},
 'flan-paca': {'response': None,
  'grammar': None,
  'completeness': None,
  'readability': None},
 'dolly_v2': {'response': '...................... 4 "Like"................................ 3 "Neutral".......................... 2 "Dislike".......................... 1 "Hate".......................

In [ ]:
l_d = 0
l_g4 = 0
l_g3 = 0
l_ll = 0
for datum in dolly:
    l_d+=len(datum['dolly_v2']['response'].split())
    l_g4+=len(datum['gpt-4']['response'].split())
    l_g3+=len(datum['gpt-3.5']['response'].split())
    l_ll+=len(datum['llama2']['response'].split())

In [ ]:
l_d//10000

109

In [ ]:
l_g4//10000

47

In [ ]:
l_g3//10000

20

In [ ]:
l_ll//10000

46

In [ ]:
for datum in dolly:
    datum['gpt-3.5'] = {'response':datum['gpr-3.5']['response'][0],
                        'grammar':None,
                        'completeness':None,
                        'readability':None}
    datum.pop('gpr-3.5')
    datum['dolly_v2']['response'] = datum['dolly_v2']['response'].replace('\t','')

In [ ]:
with open("/home/work/deeptext/yys/redpen/data/data_sampled_10k_only_eng_dolly_finished.json", "w", encoding='utf-8') as f:
    json.dump(dolly,f , ensure_ascii=False,indent=4)